In [1]:
import sys
sys.path.append("../../../")

In [2]:
import pandas as pd
from experiment.ipinyou.load import read_data
from experiment.ipinyou.agge2.train import train_estimators, train_lr_and_show_results
from experiment.measure import ProcessMeasure
import os

In [3]:
df = pd.read_json('sampling.json')

In [4]:

def _generate_space(generators):
    first_generator = generators[0]
    other_generators = generators[1:]

    if len(other_generators) == 0:
        for x in first_generator:
            yield x,
        return

    other = list(_generate_space(other_generators))
    for _first in first_generator:
        for _other in other:
            yield (_first,) + _other


def generate_space(generators, start=0, end=None):
    space = list(enumerate(list(_generate_space(generators))))
    if end is None:
        return space[start:]
    return space[start:end]


def neg_sample(df, ratio):
    clicks = df[df.click == 1]
    not_clicks = df[df.click == 0]
    return pd.concat([clicks, not_clicks.sample(int(df.shape[0] * ratio))], ignore_index=True)


if __name__ == '__main__':
    measure = ProcessMeasure()
    experiments = generate_space([
        # advertiser ids
        ['3476', '2259', '2261', '2821', '2997', '1458', '3358', '3386', '3427'],
#         ['3358', '3427'],
        # bins
        # [1, 5, 10, 50, 150, 300],
        [1, 10, 50, 150, 300],
        # re-runs
        # list(range(3)),
        list(range(30)),
        # bin type
        ['qcut'],
    ],
        # starting experiment id (you can skip start=N experiments in case of error)
        start=0)
    print(experiments)
    prev_subject = None
    df_train, df_test = (None, None)

    reuse = False

    for experiment_id, (subject, bins, attempt, bin_type) in experiments:

        if reuse and os.path.isfile('test.pckl') and os.path.isfile('train.pckl'):
            _df_train = pd.read_pickle('train.pckl')
            _df_test = pd.read_pickle('test.pckl')
        else:
            if subject != prev_subject:
                df_train, df_test = read_data(subject)
                df_train.drop(columns=['usertag'], inplace=True)
                df_test.drop(columns=['usertag'], inplace=True)

            prev_subject = subject

            test_sampling =  float(df[(df.advertiser == int(subject)) & (df.type == 'test')]['ctr']) * 2
            train_sampling = float(df[(df.advertiser == int(subject)) & (df.type == 'train')]['ctr']) * 2
            _df_test = neg_sample(df_test, test_sampling)#.sample(frac=0.1).reset_index()
            _df_train = neg_sample(df_train, train_sampling)#.sample(frac=0.1).reset_index()
#             if subject in {'1458', '3386'}:
#                 _df_test = neg_sample(df_test, 0.0005)
#                 _df_train = neg_sample(df_train, 0.0005)
#             else:
#                 _df_test = neg_sample(df_test, 0.002)
#                 _df_train = neg_sample(df_train, 0.002)

#             _df_train.to_pickle('train.pckl')
#             _df_test.to_pickle('test.pckl')

        __df_train, __df_test, estimators = train_estimators(_df_train, _df_test, None,
                                                             normalize_column="minmax",
                                                             bins=bins,
                                                             bin_type=bin_type)
    
        # experiment_output_K_L_P_ES
        fname = 'experiment_v2_output_K=A_L=1a_P=1_ES=1.json'
        new_path = '1a'
        C=None
        norm_type = None
        measure.set_suffix('_None_f={}_b={}_bt={}_r=L2'.format(__df_train.shape[1], bins, bin_type))
        lr = train_lr_and_show_results(__df_train, __df_test, None, subject + '_' + str(C) + "_" + str(bins), measure=measure, C=C,
                                       norm_type=norm_type, cnt_vec=None, f_name=fname, new_path=new_path)
        print('Done experiment id={}, adv={}, bins={}, attempt={}'.format(experiment_id, subject, bins, attempt))
        
        for C in [2, 4, 6, 8]:
#             norm_type = 'L2'
#             measure.set_suffix('_None_f={}_b={}_bt={}_r=L2'.format(__df_train.shape[1], bins, bin_type))
#             lr = train_lr_and_show_results(__df_train, __df_test, None, subject + '_' + str(C) + '_L2_' + str(bins), measure=measure, C=C,
#                                            norm_type=norm_type, cnt_vec=None, f_name=fname, new_path=new_path)
#             print('Done experiment id={}, adv={}, bins={}, attempt={}'.format(experiment_id, subject, bins, attempt))

            norm_type = 'L2+'
            measure.set_suffix('_None_f={}_b={}_bt={}_r=L2+'.format(__df_train.shape[1], bins, bin_type))
            lr = train_lr_and_show_results(__df_train, __df_test, None, subject + '_' + str(C) + '_L2+_' + str(bins), measure=measure, C=C,
                                           norm_type=norm_type, cnt_vec=None, f_name=fname, new_path=new_path)
            print('Done experiment id={}, adv={}, bins={}, attempt={}'.format(experiment_id, subject, bins, attempt))
    print('-------------------------------- RESULT --------------------------------')
    measure.print()

[(0, ('3476', 1, 0, 'qcut')), (1, ('3476', 1, 1, 'qcut')), (2, ('3476', 1, 2, 'qcut')), (3, ('3476', 1, 3, 'qcut')), (4, ('3476', 1, 4, 'qcut')), (5, ('3476', 1, 5, 'qcut')), (6, ('3476', 1, 6, 'qcut')), (7, ('3476', 1, 7, 'qcut')), (8, ('3476', 1, 8, 'qcut')), (9, ('3476', 1, 9, 'qcut')), (10, ('3476', 1, 10, 'qcut')), (11, ('3476', 1, 11, 'qcut')), (12, ('3476', 1, 12, 'qcut')), (13, ('3476', 1, 13, 'qcut')), (14, ('3476', 1, 14, 'qcut')), (15, ('3476', 1, 15, 'qcut')), (16, ('3476', 1, 16, 'qcut')), (17, ('3476', 1, 17, 'qcut')), (18, ('3476', 1, 18, 'qcut')), (19, ('3476', 1, 19, 'qcut')), (20, ('3476', 1, 20, 'qcut')), (21, ('3476', 1, 21, 'qcut')), (22, ('3476', 1, 22, 'qcut')), (23, ('3476', 1, 23, 'qcut')), (24, ('3476', 1, 24, 'qcut')), (25, ('3476', 1, 25, 'qcut')), (26, ('3476', 1, 26, 'qcut')), (27, ('3476', 1, 27, 'qcut')), (28, ('3476', 1, 28, 'qcut')), (29, ('3476', 1, 29, 'qcut')), (30, ('3476', 10, 0, 'qcut')), (31, ('3476', 10, 1, 'qcut')), (32, ('3476', 10, 2, 'qcut'

loaded df with columns = Index(['click', 'weekday', 'hour', 'bidid', 'timestamp', 'logtype',
       'ipinyouid', 'useragent', 'IP', 'region', 'city', 'adexchange',
       'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'bidprice',
       'payprice', 'keypage', 'advertiser', 'usertag', 'nclick',
       'nconversation'],
      dtype='object')
loading /Users/rwiatr/phd/make-ipinyou-data/3476/train.log.txt ... 
loaded df with columns = Index(['click', 'weekday', 'hour', 'bidid', 'timestamp', 'logtype',
       'ipinyouid', 'useragent', 'IP', 'region', 'city', 'adexchange',
       'domain', 'url', 'urlid', 'slotid', 'slotwidth', 'slotheight',
       'slotvisibility', 'slotformat', 'slotprice', 'creative', 'bidprice',
       'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')
na columns: []
new_path = 1a
Done experiment id=0, adv=3476, bins=1, attempt=0
new_path = 1a
Report: a=1, reg=2, min=0.0, 

Done experiment id=15, adv=3476, bins=1, attempt=15
new_path = 1a
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 1a
Report: a=1, reg=2, min=0.0, max=4.0
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 1a
Report: a=1, reg=4, min=0.0, max=8.0
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 1a
Report: a=1, reg=6, min=0.0, max=12.0
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 1a
Report: a=1, reg=8, min=0.0, max=16.0
Done experiment id=16, adv=3476, bins=1, attempt=16
new_path = 1a
Done experiment id=17, adv=3476, bins=1, attempt=17
new_path = 1a
Report: a=1, reg=2, min=0.0, max=4.0
Done experiment id=17, adv=3476, bins=1, attempt=17
new_path = 1a
Report: a=1, reg=4, min=0.0, max=8.0
Done experiment id=17, adv=3476, bins=1, attempt=17
new_path = 1a
Report: a=1, reg=6, min=0.0, max=12.0
Done experiment id=17, adv=3476, bins=1, attempt=17
new_path = 1a
Report: a=1, reg=8, min=0.0, max=16.0
Done experiment id=17, adv=3476, bins=1,

new_path = 1a
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 1a
Report: a=1, reg=2, min=0.0, max=4.0
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 1a
Report: a=1, reg=4, min=0.0, max=8.0
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 1a
Report: a=1, reg=6, min=0.0, max=12.0
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 1a
Report: a=1, reg=8, min=0.0, max=16.0
Done experiment id=33, adv=3476, bins=10, attempt=3
new_path = 1a
Done experiment id=34, adv=3476, bins=10, attempt=4
new_path = 1a
Report: a=1, reg=2, min=0.0, max=4.0
Done experiment id=34, adv=3476, bins=10, attempt=4
new_path = 1a
Report: a=1, reg=4, min=0.0, max=8.0
Done experiment id=34, adv=3476, bins=10, attempt=4
new_path = 1a
Report: a=1, reg=6, min=0.0, max=12.0
Done experiment id=34, adv=3476, bins=10, attempt=4
new_path = 1a
Report: a=1, reg=8, min=0.0, max=16.0
Done experiment id=34, adv=3476, bins=10, attempt=4
new_path = 1a
Done experiment id=35, adv

new_path = 1a
Done experiment id=50, adv=3476, bins=10, attempt=20
new_path = 1a
Report: a=1, reg=2, min=0.0, max=4.0
Done experiment id=50, adv=3476, bins=10, attempt=20
new_path = 1a
Report: a=1, reg=4, min=0.0, max=8.0
Done experiment id=50, adv=3476, bins=10, attempt=20
new_path = 1a
Report: a=1, reg=6, min=0.0, max=12.0
Done experiment id=50, adv=3476, bins=10, attempt=20
new_path = 1a
Report: a=1, reg=8, min=0.0, max=16.0
Done experiment id=50, adv=3476, bins=10, attempt=20
new_path = 1a
Done experiment id=51, adv=3476, bins=10, attempt=21
new_path = 1a
Report: a=1, reg=2, min=0.0, max=4.0
Done experiment id=51, adv=3476, bins=10, attempt=21
new_path = 1a
Report: a=1, reg=4, min=0.0, max=8.0
Done experiment id=51, adv=3476, bins=10, attempt=21
new_path = 1a
Report: a=1, reg=6, min=0.0, max=12.0
Done experiment id=51, adv=3476, bins=10, attempt=21
new_path = 1a
Report: a=1, reg=8, min=0.0, max=16.0
Done experiment id=51, adv=3476, bins=10, attempt=21
new_path = 1a
Done experiment 

...

	"auc_2997_6_L2+_50_None_f=156_b=50_bt=qcut_r=L2+": [54.26746485648619, 51.460437935396506],
	"delta_2997_8_L2+_50_None_f=156_b=50_bt=qcut_r=L2+": [3.9476261138916016, 3.9723029136657715],
	"auc_2997_8_L2+_50_None_f=156_b=50_bt=qcut_r=L2+": [54.10978673365761, 51.36903807436707],
	"delta_2997_None_50_None_f=145_b=50_bt=qcut_r=L2": [5.91050910949707, 4.088276147842407, 5.311838865280151, 5.111007928848267],
	"auc_2997_None_50_None_f=145_b=50_bt=qcut_r=L2": [52.02994993496891, 52.18531188198193, 52.08259839293032, 52.416840379852836],
	"delta_2997_2_L2+_50_None_f=145_b=50_bt=qcut_r=L2+": [5.233256816864014, 3.102186918258667, 3.365117073059082, 3.438770055770874],
	"auc_2997_2_L2+_50_None_f=145_b=50_bt=qcut_r=L2+": [53.843159263812424, 52.13097083400147, 52.965239546029544, 53.56993960126143],
	"delta_2997_4_L2+_50_None_f=145_b=50_bt=qcut_r=L2+": [3.1844470500946045, 2.4460301399230957, 2.3744523525238037, 2.484900712966919],
	"auc_2997_4_L2+_50_None_f=145_b=50_bt=qcut_r=L2+": [53.829084

	"delta_1458_2_L2+_50_None_f=187_b=50_bt=qcut_r=L2+": [6.7885589599609375],
	"auc_1458_2_L2+_50_None_f=187_b=50_bt=qcut_r=L2+": [63.052544179946594],
	"delta_1458_4_L2+_50_None_f=187_b=50_bt=qcut_r=L2+": [6.235398292541504],
	"auc_1458_4_L2+_50_None_f=187_b=50_bt=qcut_r=L2+": [62.97102474831819],
	"delta_1458_6_L2+_50_None_f=187_b=50_bt=qcut_r=L2+": [5.908519744873047],
	"auc_1458_6_L2+_50_None_f=187_b=50_bt=qcut_r=L2+": [62.56323888778813],
	"delta_1458_8_L2+_50_None_f=187_b=50_bt=qcut_r=L2+": [5.778063058853149],
	"auc_1458_8_L2+_50_None_f=187_b=50_bt=qcut_r=L2+": [62.92535877041523],
	"delta_1458_None_50_None_f=185_b=50_bt=qcut_r=L2": [10.67795991897583],
	"auc_1458_None_50_None_f=185_b=50_bt=qcut_r=L2": [63.29738552841386],
	"delta_1458_2_L2+_50_None_f=185_b=50_bt=qcut_r=L2+": [7.547466993331909],
	"auc_1458_2_L2+_50_None_f=185_b=50_bt=qcut_r=L2+": [64.38091464047477],
	"delta_1458_4_L2+_50_None_f=185_b=50_bt=qcut_r=L2+": [5.904157876968384],
	"auc_1458_4_L2+_50_None_f=185_b=50_bt=

	"auc_3358_None_150_None_f=187_b=150_bt=qcut_r=L2": [69.69393804653919, 74.69467911664442],
	"delta_3358_2_L2+_150_None_f=187_b=150_bt=qcut_r=L2+": [4.407812833786011, 4.877323865890503],
	"auc_3358_2_L2+_150_None_f=187_b=150_bt=qcut_r=L2+": [74.86067882021639, 76.50585445383133],
	"delta_3358_4_L2+_150_None_f=187_b=150_bt=qcut_r=L2+": [5.024013996124268, 5.454689025878906],
	"auc_3358_4_L2+_150_None_f=187_b=150_bt=qcut_r=L2+": [75.28382985030385, 77.0594338224396],
	"delta_3358_6_L2+_150_None_f=187_b=150_bt=qcut_r=L2+": [3.2828550338745117, 3.9062819480895996],
	"auc_3358_6_L2+_150_None_f=187_b=150_bt=qcut_r=L2+": [75.38091003408923, 77.21357640432785],
	"delta_3358_8_L2+_150_None_f=187_b=150_bt=qcut_r=L2+": [3.241157054901123, 4.051573991775513],
	"auc_3358_8_L2+_150_None_f=187_b=150_bt=qcut_r=L2+": [75.30754409367127, 76.68000592856083],
	"delta_3358_None_150_None_f=185_b=150_bt=qcut_r=L2": [4.875725984573364, 8.745275735855103, 9.535982847213745],
	"auc_3358_None_150_None_f=185_b=1

	"auc_3386_None_150_None_f=192_b=150_bt=qcut_r=L2": [74.14706588642711],
	"delta_3386_2_L2+_150_None_f=192_b=150_bt=qcut_r=L2+": [7.351093053817749],
	"auc_3386_2_L2+_150_None_f=192_b=150_bt=qcut_r=L2+": [75.37303623563908],
	"delta_3386_4_L2+_150_None_f=192_b=150_bt=qcut_r=L2+": [5.746652841567993],
	"auc_3386_4_L2+_150_None_f=192_b=150_bt=qcut_r=L2+": [75.53456098886517],
	"delta_3386_6_L2+_150_None_f=192_b=150_bt=qcut_r=L2+": [5.452626943588257],
	"auc_3386_6_L2+_150_None_f=192_b=150_bt=qcut_r=L2+": [75.4953804931687],
	"delta_3386_8_L2+_150_None_f=192_b=150_bt=qcut_r=L2+": [4.010631799697876],
	"auc_3386_8_L2+_150_None_f=192_b=150_bt=qcut_r=L2+": [75.22819479025796],
	"delta_3386_None_150_None_f=187_b=150_bt=qcut_r=L2": [9.817344188690186, 11.793730020523071],
	"auc_3386_None_150_None_f=187_b=150_bt=qcut_r=L2": [73.61168337103929, 73.08578000783609],
	"delta_3386_2_L2+_150_None_f=187_b=150_bt=qcut_r=L2+": [5.8169331550598145, 7.097235202789307],
	"auc_3386_2_L2+_150_None_f=187_b=15

	"delta_3427_2_L2+_50_None_f=201_b=50_bt=qcut_r=L2+": [5.88297176361084],
	"auc_3427_2_L2+_50_None_f=201_b=50_bt=qcut_r=L2+": [71.463598783013],
	"delta_3427_4_L2+_50_None_f=201_b=50_bt=qcut_r=L2+": [4.706023931503296],
	"auc_3427_4_L2+_50_None_f=201_b=50_bt=qcut_r=L2+": [71.58320438354525],
	"delta_3427_6_L2+_50_None_f=201_b=50_bt=qcut_r=L2+": [5.609225034713745],
	"auc_3427_6_L2+_50_None_f=201_b=50_bt=qcut_r=L2+": [71.5480702383889],
	"delta_3427_8_L2+_50_None_f=201_b=50_bt=qcut_r=L2+": [5.879941701889038],
	"auc_3427_8_L2+_50_None_f=201_b=50_bt=qcut_r=L2+": [71.23261046698511],
	"delta_3427_None_50_None_f=197_b=50_bt=qcut_r=L2": [6.9946489334106445, 6.909054756164551],
	"auc_3427_None_50_None_f=197_b=50_bt=qcut_r=L2": [69.72595366778049, 67.05202096088149],
	"delta_3427_2_L2+_50_None_f=197_b=50_bt=qcut_r=L2+": [6.3850321769714355, 7.081212043762207],
	"auc_3427_2_L2+_50_None_f=197_b=50_bt=qcut_r=L2+": [71.10366067891128, 68.80349547367555],
	"delta_3427_4_L2+_50_None_f=197_b=50_bt=q